In [ ]:
#r "nuget: FsToolkit.ErrorHandling, 4.16.0"

In [ ]:
open System
open FsToolkit.ErrorHandling

Let's see an example of the option data type (this is also present in Scala and Java, but not in C#). **An option has an underlying type and can hold a value of that type, or it might not have a value. So the two values could be Some() or None.**
The type of m below is Map<string, int>.

In [ ]:
let m = Map[ ("a", 1); ("b", 2) ]
let keyPresent = "a"
let keyAbsent = "c"

The return type of the TryFind function is an `option<int>`.

In [ ]:
let some = m.TryFind(keyPresent)
printfn $"Value of some: {some}"

In [ ]:
let none = m.TryFind(keyAbsent)
printfn $"Value of none: {none}"

In Scala, an absent value is represented by None (and is printed as None), but F# does not show it.

Below is how we pattern match on an optional value.

In [ ]:
match none with
  | Some(x) -> printfn $"Found key {keyAbsent}"
  | None -> printfn $"Not found key {keyAbsent}"

Careful use of Optional values helps us to reduce / eliminate NPEs in the code.

### But you might again argue, how is this pattern matching any different than if (something != null) else statement?

First, let's see how to get the value of an option safely.

In [ ]:
let someVal = Option.defaultValue 0 some
printfn $"someVal: {someVal}"

Or in an idiomatic F# way -

In [ ]:
let isomeVal = some |> Option.defaultValue 0
printfn $"isomeVal: {isomeVal}"

Now, let's see how we can use multiple option values together. This is something which is not possible in C#. But `FsToolkit.ErrorHandling` makes it a walk in the park.

In [ ]:
let tryParseInt (str: string) =
    match Int32.TryParse str with
    | (true, i) -> Some i
    | _ -> None

In [ ]:
let addAllSomes = option {
    let! x = tryParseInt "7"
    let! y = tryParseInt "5"
    let! z = tryParseInt "2"
    return x + y + z
}

printfn $"addAllSomes: {addAllSomes}"

**And if any of the values is a None, then the entire result is a None.**

In [ ]:
let addSomesWithNone = option {
    let! x = tryParseInt "7"
    let! y = tryParseInt "sad"
    let! z = tryParseInt "2"
    return x + y + z
}

printfn $"addSomesWithNone: {addSomesWithNone}"

If you have a list of string values, and you use the above function, you get a list of optional values.

In [ ]:
let listOfSomes =
    ["1"; "2"; "3"; "cannotParse"]
    |> List.map tryParseInt

printfn $"listOfSomes (which is of type list<option<int>>): {listOfSomes}"

But what if you want an option<list<int>>? Simple, use `sequenceOptionM` method.

In [ ]:
let someList = 
    ["1"; "2"; "3"]
    |> List.map tryParseInt
    |> List.sequenceOptionM

printfn $"someList (which is of type option<list<int>>): {someList}"